In [1]:
import os
import json
import wandb
import numpy as np
from glob import glob
from PIL import Image, ImageOps
import matplotlib.pyplot as plt
from tqdm.autonotebook import tqdm
from sahi.utils.file import save_json
from sahi.utils.coco import Coco, CocoCategory, CocoImage, CocoAnnotation

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  


## Save COCO Annotations

In [2]:
train_annotation_file = "./data/bdd100k/labels/bdd100k_labels_images_train.json"
with open(train_annotation_file) as json_file:
    train_annotations = json.load(json_file)
len(train_annotations)

69863

In [4]:
all_labels = []
for annotation in tqdm(train_annotations):
    for label in annotation["labels"]:
        if "box2d" in label.keys():
            all_labels.append(label["category"])

  0%|          | 0/69863 [00:00<?, ?it/s]

In [3]:
LABEL_MAP = {
    "person": 0,
    "rider": 1,
    "car": 2,
    "truck": 3,
    "bus": 4,
    "train": 5,
    "motor": 6,
    "bike": 7,
    "traffic light": 8,
    "traffic sign": 9,
}

In [4]:
def corners_to_coco(x1, y1, x2, y2, image_w, image_h):
    return [
        ((x2 + x1) / 2),
        ((y2 + y1) / 2),
        x2 - x1,
        y2 - y1
    ]

In [5]:
coco = Coco()

coco.add_category(CocoCategory(id=0, name="person"))
coco.add_category(CocoCategory(id=1, name="rider"))
coco.add_category(CocoCategory(id=2, name="car"))
coco.add_category(CocoCategory(id=3, name="truck"))
coco.add_category(CocoCategory(id=4, name="bus"))
coco.add_category(CocoCategory(id=5, name="train"))
coco.add_category(CocoCategory(id=6, name="motor"))
coco.add_category(CocoCategory(id=7, name="bike"))
coco.add_category(CocoCategory(id=8, name="traffic light"))
coco.add_category(CocoCategory(id=9, name="traffic sign"))

In [6]:
IMAGE_DIR = "./data/bdd100k/images/100k/train/"

In [7]:
for annotation in tqdm(train_annotations):
    image_file = annotation["name"]
    if os.path.isfile(os.path.join(IMAGE_DIR, image_file)):
        width, height = Image.open(os.path.join(IMAGE_DIR, image_file)).size
        coco_image = CocoImage(file_name=image_file, height=height, width=width)
        for label in annotation["labels"]:
            if "box2d" in label.keys():
                bbox = corners_to_coco(
                    label["box2d"]["x1"],
                    label["box2d"]["y1"],
                    label["box2d"]["x2"],
                    label["box2d"]["y2"],
                    width, height
                )
                category_name = label["category"]
                category_id = LABEL_MAP[category_name]
                coco_image.add_annotation(
                    CocoAnnotation(
                        bbox=bbox,
                        category_id=category_id,
                        category_name=category_name
                    )
                )
        coco.add_image(coco_image)

In [8]:
save_json(
    data=coco.json,
    save_path="./data/bdd100k/labels/coco_detection_train_100k.json"
)

## EDA on Segmentation

In [9]:
SEGMENTATION_LABEL_MAP = {
    0:  "road",
    1:  "sidewalk",
    2:  "building",
    3:  "wall",
    4:  "fence",
    5:  "pole",
    6:  "traffic light",
    7:  "traffic sign",
    8:  "vegetation",
    9:  "terrain",
    10: "sky",
    11: "person",
    12: "rider",
    13: "car",
    14: "truck",
    15: "bus",
    16: "train",
    17: "motorcycle",
    18: "bicycle",
    255: "unknown"
}

In [10]:
train_segmentation_labels = glob("./data/bdd100k/labels/sem_seg/masks/train/*")
val_segmentation_labels = glob("./data/bdd100k/labels/sem_seg/masks/val/*")
len(train_segmentation_labels), len(val_segmentation_labels)

(7000, 1000)

In [12]:
common_det_seg_files_train = []
for annotation in tqdm(train_annotations):
    image_file = annotation["name"]
    if os.path.isfile(
        os.path.join(
            "./data/bdd100k/labels/sem_seg/masks/train/",
            image_file[:-4] + ".png"
        )
    ):
        common_det_seg_files_train.append(annotation)
        len(common_det_seg_files_train)

  0%|          | 0/69863 [00:00<?, ?it/s]

In [13]:
# with wandb.init(project="bdd100k", entity="av-demo", job_type="dev"):
sample_image_file = os.path.join(IMAGE_DIR, common_det_seg_files_train[0]["name"])
sample_mask_file = os.path.join(
    "./data/bdd100k/labels/sem_seg/masks/train",
    common_det_seg_files_train[0]["name"][:-4] + ".png"
)

In [15]:
bbox_data = []
width, height = Image.open(sample_image_file).size
for label in common_det_seg_files_train[0]["labels"]:
    if "box2d" in label.keys():
        bbox_data.append({
            "position": {
                "minX": label["box2d"]["x1"] / width,
                "maxX": label["box2d"]["x2"] / width,
                "minY": label["box2d"]["y1"] / height,
                "maxY": label["box2d"]["y2"] / height
            },
            "class_id" : LABEL_MAP[label["category"]],
            "box_caption": label["category"],
        })

In [16]:
bbox_data

[{'position': {'minX': 0.97987473515625,
   'maxX': 0.99899293671875,
   'minY': 0.34803623333333333,
   'maxY': 0.3793051138888889},
  'class_id': 2,
  'box_caption': 'car'},
 {'position': {'minX': 0.9477561570312499,
   'maxX': 0.9806394632812501,
   'minY': 0.3398791333333333,
   'maxY': 0.38882172777777774},
  'class_id': 2,
  'box_caption': 'car'},
 {'position': {'minX': 0.9095197546874999,
   'maxX': 0.9523445257812501,
   'minY': 0.34531719861111115,
   'maxY': 0.41057399027777775},
  'class_id': 2,
  'box_caption': 'car'},
 {'position': {'minX': 0.8674597125,
   'maxX': 0.9316968687499999,
   'minY': 0.35211478194444445,
   'maxY': 0.42552867361111113},
  'class_id': 2,
  'box_caption': 'car'},
 {'position': {'minX': 0.64033548125,
   'maxX': 0.8101051085937501,
   'minY': 0.3602718819444445,
   'maxY': 0.5356495138888888},
  'class_id': 2,
  'box_caption': 'car'}]

In [18]:
with wandb.init(project="bdd100k", entity="av-demo", job_type="dev"):
    table = wandb.Table(columns=["Image"])
    image = np.array(Image.open(sample_image_file))
    mask = np.array(Image.open(sample_mask_file))
    image = wandb.Image(
        image,
        boxes={
            "ground_truth": {
                "box_data": bbox_data,
                "class_labels": {v: k for k, v in LABEL_MAP.items()}
            }
        },
        masks={
            "ground_truth": {
                "mask_data": mask,
                "class_labels": SEGMENTATION_LABEL_MAP
            }
        }
    )
    table.add_data(image)
    wandb.log({"BDD100k-Dataset": table})

## Save YOLOv5 Annotations

In [1]:
import os
import json
import shutil
import numpy as np
from glob import glob
from PIL import Image, ImageOps
import matplotlib.pyplot as plt
from tqdm.autonotebook import tqdm
from sahi.utils.file import save_json
from sahi.utils.coco import Coco, CocoCategory, CocoImage, CocoAnnotation

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  


In [2]:
train_annotation_file = "./data/bdd100k/labels/bdd100k_labels_images_train.json"
with open(train_annotation_file) as json_file:
    train_annotations = json.load(json_file)
len(train_annotations)

69863

In [3]:
count = 0
IMAGE_DIR = "./data/bdd100k/images/10k/train/"
for annotation in tqdm(train_annotations):
    image_file = annotation["name"]
    if os.path.isfile(os.path.join(IMAGE_DIR, image_file)):
        for label in annotation["labels"]:
            if "box2d" in label.keys():
                count += 1
                break
count

  0%|          | 0/69863 [00:00<?, ?it/s]

2972

In [4]:
def corners_to_yolo(x1, y1, x2, y2, image_w, image_h):
    return [
        ((x2 + x1) / (2 * image_w)),
        ((y2 + y1) / (2 * image_h)),
        (x2 - x1) / image_w,
        (y2 - y1) / image_h
    ]

In [5]:
LABEL_MAP = {
    "person": 0,
    "rider": 1,
    "car": 2,
    "truck": 3,
    "bus": 4,
    "train": 5,
    "motor": 6,
    "bike": 7,
    "traffic light": 8,
    "traffic sign": 9,
}

In [6]:
all_10k_annotated_images = []
for annotation in tqdm(train_annotations):
    image_file = annotation["name"]
    if os.path.isfile(os.path.join(IMAGE_DIR, image_file)):
        all_10k_annotated_images.append(image_file)
        width, height = Image.open(os.path.join(IMAGE_DIR, image_file)).size
        current_labels = ""
        for label in annotation["labels"]:
            if "box2d" in label.keys():
                bbox = corners_to_yolo(
                    label["box2d"]["x1"],
                    label["box2d"]["y1"],
                    label["box2d"]["x2"],
                    label["box2d"]["y2"],
                    width, height
                )
                category_name = label["category"]
                category_id = LABEL_MAP[category_name]
                current_labels += " ".join(str(i) for i in [category_id] + bbox) + "\n"
        # print(current_labels)
        label_file_name = image_file.split(".")[0] + ".txt"
        with open(os.path.join("./data/bdd100k-yolov5/labels", label_file_name), "w") as f:
            f.write(current_labels)

  0%|          | 0/69863 [00:00<?, ?it/s]

In [7]:
all_annotations = glob("./data/bdd100k-yolov5/labels/*")
len(all_10k_annotated_images), len(all_annotations)

(2972, 2972)

In [8]:
for filename in tqdm(all_10k_annotated_images):
    shutil.copyfile(
        os.path.join(IMAGE_DIR, filename),
        os.path.join("./data/bdd100k-yolov5/images", filename)
    )

  0%|          | 0/2972 [00:00<?, ?it/s]

In [9]:
all_images = glob("./data/bdd100k-yolov5/images/*")
len(all_images), len(all_annotations)

(2972, 2972)

In [10]:
from sklearn.model_selection import train_test_split

train_images, val_images, train_labels, val_labels = train_test_split(
    sorted(all_images), sorted(all_annotations), test_size=0.3, random_state=42
)
val_images, test_images, val_labels, test_labels = train_test_split(
    sorted(val_images), sorted(val_labels), test_size=0.4, random_state=42
)

In [11]:
print(len(train_images), len(train_labels))
print(len(val_images), len(val_labels))
print(len(test_images), len(test_labels))

2080 2080
535 535
357 357


In [12]:
for idx in tqdm(range(len(train_images))):
    shutil.copyfile(
        train_images[idx],
        os.path.join(
            "./data/bdd100k-yolov5/train/images",
            train_images[idx].split("/")[-1]
        )
    )

    
for idx in tqdm(range(len(train_labels))):
    shutil.copyfile(
        train_labels[idx],
        os.path.join(
            "./data/bdd100k-yolov5/train/labels",
            train_labels[idx].split("/")[-1]
        )
    )

  0%|          | 0/2080 [00:00<?, ?it/s]

  0%|          | 0/2080 [00:00<?, ?it/s]

In [13]:
for idx in tqdm(range(len(val_images))):
    shutil.copyfile(
        val_images[idx],
        os.path.join(
            "./data/bdd100k-yolov5/val/images",
            val_images[idx].split("/")[-1]
        )
    )

    
for idx in tqdm(range(len(val_labels))):
    shutil.copyfile(
        val_labels[idx],
        os.path.join(
            "./data/bdd100k-yolov5/val/labels",
            val_labels[idx].split("/")[-1]
        )
    )

  0%|          | 0/535 [00:00<?, ?it/s]

  0%|          | 0/535 [00:00<?, ?it/s]

In [14]:
for idx in tqdm(range(len(test_images))):
    shutil.copyfile(
        test_images[idx],
        os.path.join(
            "./data/bdd100k-yolov5/test/images",
            test_images[idx].split("/")[-1]
        )
    )

    
for idx in tqdm(range(len(test_labels))):
    shutil.copyfile(
        test_labels[idx],
        os.path.join(
            "./data/bdd100k-yolov5/test/labels",
            test_labels[idx].split("/")[-1]
        )
    )

  0%|          | 0/357 [00:00<?, ?it/s]

  0%|          | 0/357 [00:00<?, ?it/s]

## EDA on Segmentation

In [1]:
import os
import json
import shutil
import numpy as np
from glob import glob
from PIL import Image, ImageOps
import matplotlib.pyplot as plt
from tqdm.autonotebook import tqdm

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  


In [4]:
train_images_10_k = glob("./data/bdd100k/images/10k/train/*")
train_labels_10_k = glob("./data/bdd100k/labels/sem_seg/masks/train/*")

In [5]:
len(train_images_10_k), len(train_labels_10_k)

(7000, 7000)

In [8]:
count = 0
for idx in range(7000):
    if train_images_10_k[idx].split("/")[-1].split(".")[0] == train_labels_10_k[idx].split("/")[-1].split(".")[0]:
        count += 1


count

1